In [1]:
!pip install faiss-cpu sentence-transformers gradio

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 22.4 MB/s  0:00:00

   ---------------------------------------- 0/3 [widgetsnbextension]
   ------------- -------------------------- 1/3 [jupyterlab_widgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- 

In [8]:
import sqlite3
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import ollama
import re
import gradio as gr

In [4]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE customers (
    id INTEGER PRIMARY KEY,
    name TEXT,
    signup_date TEXT
)
""")

cursor.executemany("""
INSERT INTO customers (name, signup_date) VALUES (?, ?)
""", [
    ("bp", "2025-08-12"),
    ("juvvadi", "2025-08-20"),
    ("prasad", "2025-09-01"),
])
conn.commit()

In [5]:
kb_docs = [
    "The signup process requires filling out a registration form.",
    "Customers receive a confirmation email after signing up.",
    "Password resets can be done via the account settings page."
]

embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(kb_docs)

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

In [6]:
class DataQnAAgent:
    def __init__(self, conn, kb_docs, kb_index, embedder, model="llama3"):
        self.conn = conn
        self.kb_docs = kb_docs
        self.kb_index = kb_index
        self.embedder = embedder
        self.model = model

    def classify_query(self, query: str) -> str:
        db_keywords = ["count", "list", "show", "table", "record", "id", "how many", "average", "sum"]
        if any(kw in query.lower() for kw in db_keywords):
            return "DB"
        return "KB"

    def generate_sql(self, query: str) -> str:
        prompt = f"""
        You are a SQL expert. Convert this natural language request into a valid SQLite query.
        Database schema:
        TABLE customers(id INTEGER, name TEXT, signup_date TEXT)
        
        Question: {query}
        SQL:
        """
        response = ollama.chat(model=self.model, messages=[{"role": "user", "content": prompt}])
        sql = response["message"]["content"].strip()
        match = re.search(r"(SELECT|INSERT|UPDATE|DELETE).*?;", sql, re.IGNORECASE | re.DOTALL)
        if match:
            return match.group(0)
        else:
            raise ValueError(f"Could not parse SQL from response: {sql}")
    def answer_db(self, query: str):
        try:
            sql = self.generate_sql(query)
            result = self.conn.execute(sql).fetchall()
            return f"SQL: {sql}\nResult: {result}"
        except Exception as e:
            return f"Failed to execute SQL: {e}"

    def answer_kb(self, query: str):
        q_emb = self.embedder.encode([query])
        D, I = self.kb_index.search(np.array(q_emb), k=1)
        context = self.kb_docs[I[0][0]]

        prompt = f"""
        You are a helpful assistant. Use the following context to answer the question.

        Context: {context}

        Question: {query}
        Answer:
        """
        response = ollama.chat(model=self.model, messages=[{"role": "user", "content": prompt}])
        return response["message"]["content"].strip()

    def answer(self, query: str):
        route = self.classify_query(query)
        if route == "DB":
            return {"route": "DB", "answer": self.answer_db(query)}
        else:
            return {"route": "KB", "answer": self.answer_kb(query)}

In [9]:
agent = DataQnAAgent(conn, kb_docs, index, embedder)

def chat_with_agent(query):
    response = agent.answer(query)
    return f"Route: {response['route']} Answer: {response['answer']}"

iface = gr.Interface(
    fn=chat_with_agent,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
    outputs="text",
    title="Data Q&A Agent",
    description="Ask questions about the database or knowledge base."
)

if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
